# Run the following 4 cells and run the prediction as per your discharge summary data

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_excel("/content/drive/MyDrive/Clinical_Dataset.xlsx")

In [ ]:
import pandas as pd

# Sample DataFrame with a column 'text_column' containing strings


# Function to remove the first two and last character of a string
def trim_string(s):
    if len(s) > 3:  # Ensure the string is long enough to remove characters
        return s[2:-1]
    else:
        return ''  # Return empty if the string is too short

# Apply the function to each row in the 'text_column'
df['processed_column'] = df['entities'].apply(trim_string)


In [ ]:
import pandas as pd
import json

# Sample DataFrame setup
# Assuming df is initialized with a column 'processed_column' containing JSON-like strings

# Initialize an empty dictionary to hold the universal name-to-category mapping
universal_name_category_map = {}

# Initialize an empty list to hold all names
universal_name_list = []

def to_lower(text):
    return text.lower()  # Converts text to lowercase

def extract_summary(entities_str):
    # Replace 'None' with 'null' for JSON compatibility
    entities_str = entities_str.replace('None', 'null')

    try:
        # Convert the string to a Python list
        data = json.loads(entities_str)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return [], "", {}  # Return empty list, empty string, and empty dictionary in case of error

    # Create a summary with only name, category, start, and end
    summary = [{"start": entry[4], "end": entry[5], "name": to_lower(entry[1]), "category": to_lower(entry[2])} for entry in data]

    # Collect names and concatenate them into a string
    names = [to_lower(entry[1]) for entry in data]
    names_string = ", ".join(names)

    # Create a dictionary mapping names to their categories for this row
    name_category_map = {to_lower(entry[1]): to_lower(entry[2]) for entry in data}

    # Update the universal mapping and universal name list
    for name, category in name_category_map.items():
        universal_name_category_map[name] = category
        universal_name_list.append(name)  # Add name to the universal name list

    return summary, names_string

# Sample DataFrame initialization for demonstration purposes
# df = pd.DataFrame(...)  # Make sure df is properly initialized with your data

# Apply the function to each row in the 'processed_column'
df['summary'], df['name'] = zip(*df['processed_column'].apply(extract_summary))

# universal_name_category_map now contains the mapping of all names to their categories across the DataFrame
print("Universal Name-Category Map:", universal_name_category_map)
print("Universal Name List:", universal_name_list)


Error decoding JSON: Expecting ',' delimiter: line 1 column 32765 (char 32764)
Error decoding JSON: Expecting ',' delimiter: line 1 column 32765 (char 32764)
Error decoding JSON: Unterminated string starting at: line 1 column 32760 (char 32759)
Error decoding JSON: Expecting ',' delimiter: line 1 column 32765 (char 32764)
Error decoding JSON: Unterminated string starting at: line 1 column 32763 (char 32762)
Error decoding JSON: Unterminated string starting at: line 1 column 32758 (char 32757)
Error decoding JSON: Invalid \escape: line 1 column 13868 (char 13867)
Error decoding JSON: Expecting ',' delimiter: line 1 column 32765 (char 32764)
Error decoding JSON: Expecting ',' delimiter: line 1 column 32765 (char 32764)
Error decoding JSON: Invalid \escape: line 1 column 19549 (char 19548)
Error decoding JSON: Expecting ',' delimiter: line 1 column 32765 (char 32764)
Error decoding JSON: Expecting ',' delimiter: line 1 column 32765 (char 32764)
Error decoding JSON: Expecting ',' delimiter

In [ ]:
df1=pd.DataFrame()
df_cleaned = df[df['summary'].apply(lambda x: len(x) > 0)]
df1["summary"]=df_cleaned["name"]
df1["text"]=df_cleaned["Discharge Summary"]

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset, Features, Sequence, Value

# Load and preprocess your data
df = df1

# Initialize the tokenizer and model
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples, tokenizer, max_input_len=512, max_target_len=128):
    inputs = examples['text']
    targets = examples['summary']

    input_chunks = [chunk_text(text, tokenizer, max_len=max_input_len) for text in inputs]
    target_chunks = [chunk_text(text, tokenizer, max_len=max_target_len) for text in targets]

    # Flatten the chunks and align lengths
    flattened_inputs = [chunk for sublist in input_chunks for chunk in sublist]
    flattened_targets = [chunk for sublist in target_chunks for chunk in sublist]

    # Adjust padding lengths
    max_input_length = max(len(ids) for ids in flattened_inputs)
    max_target_length = max(len(ids) for ids in flattened_targets)

    # Pad sequences
    input_ids = [ids + [tokenizer.pad_token_id] * (max_input_length - len(ids)) for ids in flattened_inputs]
    labels = [lbls + [-100] * (max_input_length - len(lbls)) for lbls in flattened_targets]

    return {
        'input_ids': input_ids,
        'labels': labels
    }


In [ ]:
from datasets import Dataset

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'summary']])
import pandas as pd
from transformers import T5Tokenizer
from datasets import Dataset

# Sample DataFrame
df = df1

# Initialize the tokenizer
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)

def preprocess_function(examples, tokenizer, max_input_len=512, max_target_len=128):
    inputs = tokenizer(examples['text'], max_length=max_input_len, truncation=True, padding='max_length')
    targets = tokenizer(examples['summary'], max_length=max_target_len, truncation=True, padding='max_length')

    return {
        'input_ids': inputs['input_ids'],
        'labels': targets['input_ids']
    }

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'summary']])

# Apply preprocessing
def preprocess_batch(batch):
    return preprocess_function(batch, tokenizer)

dataset = dataset.map(preprocess_batch, batched=True, remove_columns=['text', 'summary', '__index_level_0__'])

# Check the structure
print(dataset.features)
print(dataset[0])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/334 [00:00<?, ? examples/s]

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
{'input_ids': [3, 24090, 3288, 2938, 3341, 3, 14100, 3, 4327, 3420, 2079, 4241, 489, 20173, 13523, 3264, 25095, 10, 1206, 5422, 445, 26296, 4652, 24786, 4666, 276, 13570, 6365, 15, 26, 3, 15438, 22100, 7678, 3, 10, 10668, 20173, 13523, 3264, 3750, 19318, 3, 10, 6365, 15, 26, 2678, 7993, 7678, 3, 10, 10668, 87, 2688, 13523, 3264, 6045, 3162, 3597, 28309, 10826, 8418, 7400, 14408, 3, 10, 1844, 3, 17, 21783, 4310, 32, 2258, 12398, 3, 5, 6045, 3162, 3597, 28309, 6828, 254, 2326, 18290, 3, 10, 37, 1868, 141, 3, 9, 18012, 7496, 12682, 1016, 3689, 30, 10668, 13311, 12989, 3264, 3, 6, 8, 1868, 141, 46, 9241, 26, 9709, 3689, 30, 10668, 87, 2266, 87, 3264, 3, 5, 3, 7094, 22633, 476, 3347, 3, 22120, 6431, 3, 10781, 16560, 134, 3, 10, 37, 1868, 19, 3, 9, 7123, 215, 625, 3955, 28, 442, 3, 17, 21783, 4310, 32, 2258, 12398, 24, 6935, 30, 8, 

In [ ]:
# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'summary']])


In [ ]:
# Apply preprocessing
def preprocess_batch(batch):
    return preprocess_function(batch, tokenizer)

# Use batched=True if your preprocessing function can handle batching
dataset = dataset.map(preprocess_batch, batched=True, remove_columns=['text', 'summary'])


Map:   0%|          | 0/334 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict

# Assuming `dataset` is your full dataset
# Split the dataset into training and evaluation datasets
dataset = dataset.shuffle(seed=42)  # Shuffle for randomness
train_test_split = dataset.train_test_split(test_size=0.1)  # 10% for validation

# Create a DatasetDict
datasets = DatasetDict({
    'train': train_test_split['train'],
    'eval': train_test_split['test']
})

# Update training_args to use 'train' and 'eval'
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",          # update from evaluation_strategy to eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize Trainer with updated dataset names
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],  # use 'train' from dataset dict
    eval_dataset=datasets['eval'],    # use 'eval' from dataset dict
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,3.074800,1.967908
2,2.285700,1.642564
3,2.004700,1.500211
4,1.896300,1.421290
5,1.800300,1.363458
6,1.723900,1.327395
7,1.526500,1.301394
8,1.586200,1.285671
9,1.535100,1.276064
10,1.562600,1.272614


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=380, training_loss=1.9738129841653924, metrics={'train_runtime': 241.0789, 'train_samples_per_second': 12.444, 'train_steps_per_second': 1.576, 'total_flos': 406025404416000.0, 'train_loss': 1.9738129841653924, 'epoch': 10.0})

In [ ]:
# Save the model
model.save_pretrained('./fine-tuned-t5')
tokenizer.save_pretrained('./fine-tuned-t5')

# Define the save path on Google Drive
save_path = '/content/drive/MyDrive/discharge_summary_model'

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
# Evaluate the model
results = trainer.evaluate()
print(results)


{'eval_loss': 1.2726136445999146, 'eval_runtime': 0.5685, 'eval_samples_per_second': 59.81, 'eval_steps_per_second': 8.796, 'epoch': 10.0}


In [ ]:
def chunk_text(text, tokenizer, max_len=512):
    tokens = tokenizer.encode(text, truncation=False)
    chunks = [tokens[i:i + max_len] for i in range(0, len(tokens), max_len)]
    return chunks


In [ ]:
def generate_summary(text, model, tokenizer, max_input_len=512, max_target_len=128):
    chunks = chunk_text(text, tokenizer, max_len=max_input_len)

    generated_texts = []
    for chunk in chunks:
        input_ids = torch.tensor(chunk).unsqueeze(0)
        attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_target_len,
                num_beams=5,
                early_stopping=True
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(generated_text)

    return ' '.join(generated_texts)

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('./fine-tuned-t5')
tokenizer = T5Tokenizer.from_pretrained('./fine-tuned-t5')

# Example usage
text_to_summarize = df1["text"][1]
generated_summary = generate_summary(text_to_summarize, model, tokenizer)
print(f"Generated Summary: {generated_summary}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (1260 > 512). Running this sequence through the model will result in indexing errors


Generated Summary: the patient is a 28-year-old woman who is HIV positive for two years. she presented with left upper quadrant pain as well as nausea and vomiting which is a long-standing complaint. she was diagnosed in 1991 during the birth of her child. she claims she does not know why she is HIV positive. she is from Maryland, apparently had no blood transfusions before the birth of her children so it is presumed heterosexual transmission. Percocet, Prinovil, Dapsone, Mycelex troches, BACTRIM, ALLERGIES, BACTRIM, MEDICATIONS, Percocet, Prinovil, Dapsone, Mycelex troches, afebrile, orthostatic, blood pressure, 110/80, HEENT exam, normal limits, auscultation, percussion bilaterally, cardiovascular exam, regular rate, rhythm, murmur, abdomen, soft, nontender the patient was discharged to home in stable condition. cultures were pending on her aspirate and will be treated with po doxycycline.


In [ ]:
# Example comma-separated string
# thoracotomy syndrome, thoracotomy incision, thoracic aortic aneurysm, neuropathic pain, incision, Vicodin, one to two tablets, every four hours, p.r.n., Fentanyl patch, 25 mcg an hour, change of patch, 72 hours, Elavil, 50 mgq,h.s., Neurontin, 600 mg, p.o., t.i.d. atrovent nebulizer, hemidi
# comma_separated_string =gennerated_summary

# Convert to a list
list_of_items = generated_summary.split(",")

# Print the list
print(list_of_items)


['the patient is a 28-year-old woman who is HIV positive for two years. she presented with left upper quadrant pain as well as nausea and vomiting which is a long-standing complaint. she was diagnosed in 1991 during the birth of her child. she claims she does not know why she is HIV positive. she is from Maryland', ' apparently had no blood transfusions before the birth of her children so it is presumed heterosexual transmission. Percocet', ' Prinovil', ' Dapsone', ' Mycelex troches', ' BACTRIM', ' ALLERGIES', ' BACTRIM', ' MEDICATIONS', ' Percocet', ' Prinovil', ' Dapsone', ' Mycelex troches', ' afebrile', ' orthostatic', ' blood pressure', ' 110/80', ' HEENT exam', ' normal limits', ' auscultation', ' percussion bilaterally', ' cardiovascular exam', ' regular rate', ' rhythm', ' murmur', ' abdomen', ' soft', ' nontender the patient was discharged to home in stable condition. cultures were pending on her aspirate and will be treated with po doxycycline.']


In [ ]:
len(list_of_items)

28

In [ ]:
import re

# Example list of strings
string_list = list_of_items

# Example list of words/phrases to search for
word_list = universal_name_list
# List to store the matched words/phrases
matched_words = []

# Iterate over each word/phrase in word_list
for word in word_list:
    # Check if the word/phrase is in any string in string_list using regular expressions
    if any(re.search(r'\b' + re.escape(word) + r'\b', text, re.IGNORECASE) for text in string_list):
        matched_words.append(word)

# Output the list of matched words/phrases
print(matched_words)


['pain', 'two', 'blood pressure', 'heent', 'cardiovascular', 'regular', 'normal', 'abdomen', 'nontender', 'positive', 'rate', '80', 'hiv positive', 'left upper quadrant', 'pain', 'nausea', 'vomiting', 'complaint', 'hiv', 'blood transfusions', 'normal', 'bactrim', 'percocet', 'prinovil', 'dapsone', 'mycelex', 'troches', 'stable', 'afebrile', 'blood pressure', '110/80', 'heent exam', 'cardiovascular exam', 'regular', 'murmur', 'abdomen', 'nontender', 'positive', 'cultures', 'po', 'doxycycline', 'aspirate', 'po', 'pain', 'stable', 'murmur', 'regular', 'vomiting', 'medications', 'po', 'pain', 'normal', 'po', 'percocet', 'transfusions', 'blood pressure', 'abdomen', 'normal', 'cultures', 'pain', 'po', 'percocet', 'po', 'nausea', 'pain', 'afebrile', 'blood pressure', 'regular', 'abdomen', 'percocet', 'normal', 'medications', 'pain', 'heent', 'normal', 'afebrile', 'medications', 'po', 'rhythm', 'afebrile', 'blood pressure', '80', 'heent', 'cardiovascular', 'abdomen', 'normal', 'pain', 'culture

In [ ]:
import re
from collections import Counter

# Sample text from the DataFrame
text = df1["text"][0]

# List of words to find
words_to_find = matched_words

# Dictionary with additional information for each word
word_info = universal_name_category_map

# Remove extra spaces, convert to lowercase, and create a list of processed words
stripped_and_lowered_words = [word.strip().lower() for word in words_to_find]

# Convert text to lowercase for case-insensitive search
text_lower = text.lower()

# Count the frequency of each word in the text
word_counts = Counter(re.findall(r'\b\w+\b', text_lower))

# Find the words, their spans, and include dictionary information and list status
words_with_info = []
for word in stripped_and_lowered_words:
    for match in re.finditer(r'\b{}\b'.format(re.escape(word)), text_lower):
        span = match.span()
        info = word_info.get(word, "No info available")
        # Determine list status based on frequency
        list_status = "likely"
        words_with_info.append([
            word,
            info,
            str(span[0]),
            str(span[1]),
            [list_status]
        ])

# Print the result
for item in words_with_info:
    print(item)


In [ ]:
# universal_name_category_map

In [ ]:
import re
from collections import Counter

# Sample text from the DataFrame
text = df1["text"][0]

# List of words to find
words_to_find = matched_words

# Dictionary with additional information for each word
word_info = universal_name_category_map

# Remove extra spaces, convert to lowercase, and create a list of processed words
stripped_and_lowered_words = [word.strip().lower() for word in words_to_find]

# Convert text to lowercase for case-insensitive search
text_lower = text.lower()


# List to store words with their info and spans
words_with_info = []

# Find the words, their spans, and include dictionary information
for word in words_to_find:
    word_lower = word.lower()  # Ensure word is lowercase for matching

    # Find all occurrences of the word in the text
    for match in re.finditer(r'\b{}\b'.format(re.escape(word_lower)), text_lower):
        span = match.span()
        info = universal_name_category_map.get(word_lower, "No info available")
        words_with_info.append([
            word,
            info,
            span[0],
            span[1]
        ])

# Print the result
for item in words_with_info:
    print(item)


In [ ]:
from collections import Counter

# Sample data with span as [start, end]
data =words_with_info

# Step 1: Count occurrences of each word in the original data
word_count = Counter(item[0] for item in data)

# Step 2: Keep only up to three entries per word
limited_entries = []
word_occurrences = Counter()

for item in data:
    word = item[0]
    if word_occurrences[word] < 3:
        limited_entries.append(item)
        word_occurrences[word] += 1

# Step 3: Add 'likely' or 'unlikely' based on the count in the original data
data_with_likelihood = [
    [item[0], item[1], item[2], item[3], ['likely'] if word_count[item[0]] > 1 else ['unlikely']]
    for item in limited_entries
]

# Step 4: Sort the list based on the 'end' value
sorted_data = sorted(data_with_likelihood, key=lambda x: x[3])

print(sorted_data)


[['pain', 'problem', 57, 61, ['likely']], ['in', 'bm_unit', 529, 531, ['likely']], ['pain', 'problem', 580, 584, ['likely']], ['two', 'med_dose', 645, 648, ['likely']], ['as', 'problem', 827, 829, ['likely']], ['pain', 'problem', 856, 860, ['likely']], ['as', 'problem', 873, 875, ['likely']], ['as', 'problem', 883, 885, ['likely']], ['two', 'med_dose', 1227, 1230, ['likely']], ['in', 'bm_unit', 1309, 1311, ['likely']], ['medications', 'medicine', 1350, 1361, ['likely']], ['medications', 'medicine', 1350, 1361, ['likely']], ['medications', 'medicine', 1350, 1361, ['likely']], ['pressure', 'body_measurement', 1589, 1597, ['likely']], ['pressure', 'body_measurement', 1589, 1597, ['likely']], ['before', 'med_frequency', 1782, 1788, ['unlikely']], ['two', 'med_dose', 2152, 2155, ['likely']], ['allergies', 'problem', 2248, 2257, ['likely']], ['allergies', 'problem', 2248, 2257, ['likely']], ['allergies', 'problem', 2248, 2257, ['likely']], ['in', 'bm_unit', 2308, 2310, ['likely']], ['no', 'b

In [ ]:
type(words_with_info)

list

# prediction

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Define the save path on Google Drive
save_path = '/content/drive/MyDrive/discharge_summary_model'

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(save_path)
tokenizer = T5Tokenizer.from_pretrained(save_path)

def chunk_text(text, tokenizer, max_len=512):
    tokens = tokenizer.encode(text, truncation=False)
    chunks = [tokens[i:i + max_len] for i in range(0, len(tokens), max_len)]
    return chunks

def generate_summary(text, model, tokenizer, max_input_len=512, max_target_len=128):
    chunks = chunk_text(text, tokenizer, max_len=max_input_len)

    generated_texts = []
    for chunk in chunks:
        input_ids = torch.tensor(chunk).unsqueeze(0)
        attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_target_len,
                num_beams=5,
                early_stopping=True
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(generated_text)

    return ' '.join(generated_texts)

# Example usage
text_to_summarize = df1["text"][1]  # Replace with the actual text you want to summarize



generated_summary = generate_summary(text_to_summarize, model, tokenizer)
print(f"Generated Summary: {generated_summary}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (1260 > 512). Running this sequence through the model will result in indexing errors


Generated Summary: the patient is a 28-year-old woman who is HIV positive for two years. she presented with left upper quadrant pain as well as nausea and vomiting which is a long-standing complaint. she was diagnosed in 1991 during the birth of her child. she claims she does not know why she is HIV positive. she is from Maryland, apparently had no blood transfusions before the birth of her children so it is presumed heterosexual transmission. Percocet, Prinovil, Dapsone, Mycelex troches, BACTRIM, ALLERGIES, BACTRIM, MEDICATIONS, Percocet, Prinovil, Dapsone, Mycelex troches, afebrile, orthostatic, blood pressure, 110/80, HEENT exam, normal limits, auscultation, percussion bilaterally, cardiovascular exam, regular rate, rhythm, murmur, abdomen, soft, nontender the patient was discharged to home in stable condition. cultures were pending on her aspirate and will be treated with po doxycycline.


In [ ]:
list_of_items = generated_summary.split(",")

# Print the list
print(list_of_items)

['the patient is a 28-year-old woman who is HIV positive for two years. she presented with left upper quadrant pain as well as nausea and vomiting which is a long-standing complaint. she was diagnosed in 1991 during the birth of her child. she claims she does not know why she is HIV positive. she is from Maryland', ' apparently had no blood transfusions before the birth of her children so it is presumed heterosexual transmission. Percocet', ' Prinovil', ' Dapsone', ' Mycelex troches', ' BACTRIM', ' ALLERGIES', ' BACTRIM', ' MEDICATIONS', ' Percocet', ' Prinovil', ' Dapsone', ' Mycelex troches', ' afebrile', ' orthostatic', ' blood pressure', ' 110/80', ' HEENT exam', ' normal limits', ' auscultation', ' percussion bilaterally', ' cardiovascular exam', ' regular rate', ' rhythm', ' murmur', ' abdomen', ' soft', ' nontender the patient was discharged to home in stable condition. cultures were pending on her aspirate and will be treated with po doxycycline.']


In [ ]:
import re
from collections import Counter

# Example lists
string_list = list_of_items  # List of strings to search in
word_list = universal_name_list  # List of words/phrases to search for
word_info = universal_name_category_map  # Dictionary with additional info for each word

# Step 1: Find matched words/phrases
matched_words = []
for word in word_list:
    if any(re.search(r'\b' + re.escape(word) + r'\b', text, re.IGNORECASE) for text in string_list):
        matched_words.append(word)

# Remove duplicates from matched_words
matched_words = list(set(matched_words))

# Sample text from the DataFrame
text = df1["text"][0]

# Remove extra spaces, convert to lowercase, and create a list of processed words
stripped_and_lowered_words = [word.strip().lower() for word in matched_words]

# Convert text to lowercase for case-insensitive search
text_lower = text.lower()

# List to store words with their info and spans
words_with_info = []

# Find the words, their spans, and include dictionary information
for word in stripped_and_lowered_words:
    word_lower = word.lower()  # Ensure word is lowercase for matching

    # Find all occurrences of the word in the text
    for match in re.finditer(r'\b{}\b'.format(re.escape(word_lower)), text_lower):
        span = match.span()
        info = word_info.get(word_lower, "No info available")
        words_with_info.append([
            word,
            info,
            span[0],
            span[1]
        ])

# Step 2: Count occurrences of each word in the original data
word_count = Counter(item[0] for item in words_with_info)

# Step 3: Keep only up to three entries per word
limited_entries = []
word_occurrences = Counter()

for item in words_with_info:
    word = item[0]
    if word_occurrences[word] < 3:
        limited_entries.append(item)
        word_occurrences[word] += 1

# Step 4: Add 'likely' or 'unlikely' based on the count in the original data
data_with_likelihood = [
    [item[0], item[1], item[2], item[3], 'likely' if word_count[item[0]] > 1 else 'unlikely']
    for item in limited_entries
]

# Step 5: Sort the list based on the 'end' value
sorted_data = sorted(data_with_likelihood, key=lambda x: x[3])

# Step 6: Remove duplicates from sorted_data
seen = set()
unique_sorted_data = []
for item in sorted_data:
    entry_tuple = tuple(item[:4])  # Exclude likelihood from tuple for uniqueness check
    if entry_tuple not in seen:
        seen.add(entry_tuple)
        unique_sorted_data.append(item)

# Convert unique sorted data to list of dictionaries
final_output = [
    {
        'name': item[0],
        'category': item[1],
        'start': item[2],
        'end': item[3]
    }
    for item in unique_sorted_data
]

# Print the final output
print("Unique sorted data with likelihood (in dictionary format):")
for entry in final_output:
    print(entry)


Unique sorted data with likelihood (in dictionary format):
{'name': 'pain', 'category': 'problem', 'start': 57, 'end': 61}
{'name': 'in', 'category': 'bm_unit', 'start': 529, 'end': 531}
{'name': 'pain', 'category': 'problem', 'start': 580, 'end': 584}
{'name': 'two', 'category': 'med_dose', 'start': 645, 'end': 648}
{'name': 'as', 'category': 'problem', 'start': 827, 'end': 829}
{'name': 'pain', 'category': 'problem', 'start': 856, 'end': 860}
{'name': 'as', 'category': 'problem', 'start': 873, 'end': 875}
{'name': 'as', 'category': 'problem', 'start': 883, 'end': 885}
{'name': 'two', 'category': 'med_dose', 'start': 1227, 'end': 1230}
{'name': 'in', 'category': 'bm_unit', 'start': 1309, 'end': 1311}
{'name': 'medications', 'category': 'medicine', 'start': 1350, 'end': 1361}
{'name': 'pressure', 'category': 'body_measurement', 'start': 1589, 'end': 1597}
{'name': 'before', 'category': 'med_frequency', 'start': 1782, 'end': 1788}
{'name': 'two', 'category': 'med_dose', 'start': 2152, '

# code for dataframe


In [16]:
##give your dataframe


df1 = pd.DataFrame({
    "text": ["Example text to summarize.", "Another text to summarize."]
})
column_name="text"

In [17]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import re
from collections import Counter

# Define the save path on Google Drive
save_path = '/content/drive/MyDrive/discharge_summary_model'

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(save_path)
tokenizer = T5Tokenizer.from_pretrained(save_path)

def chunk_text(text, tokenizer, max_len=512):
    tokens = tokenizer.encode(text, truncation=False)
    chunks = [tokens[i:i + max_len] for i in range(0, len(tokens), max_len)]
    return chunks

def generate_summary(text, model, tokenizer, max_input_len=512, max_target_len=128):
    chunks = chunk_text(text, tokenizer, max_len=max_input_len)

    generated_texts = []
    for chunk in chunks:
        input_ids = torch.tensor(chunk).unsqueeze(0)
        attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_target_len,
                num_beams=5,
                early_stopping=True
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(generated_text)

    return ' '.join(generated_texts)



text_to_summarize = df1[column_name][0]  # Replace with the actual text you want to summarize
generated_summary = generate_summary(text_to_summarize, model, tokenizer)
print(f"Generated Summary: {generated_summary}")

# Example lists for word processing
string_list = df1[column_name].tolist()  # List of strings to search in
word_list = universal_name_list  # List of words/phrases to search for
word_info = universal_name_category_map  # Dictionary with additional info for each word

# Step 1: Find matched words/phrases
matched_words = []
for word in word_list:
    if any(re.search(r'\b' + re.escape(word) + r'\b', text, re.IGNORECASE) for text in string_list):
        matched_words.append(word)

# Sample text from the DataFrame
text = df1[column_name][0]

# Remove extra spaces, convert to lowercase, and create a list of processed words
stripped_and_lowered_words = [word.strip().lower() for word in matched_words]

# Convert text to lowercase for case-insensitive search
text_lower = text.lower()

# List to store words with their info and spans
words_with_info = []

# Find the words, their spans, and include dictionary information
for word in stripped_and_lowered_words:
    word_lower = word.lower()  # Ensure word is lowercase for matching

    # Find all occurrences of the word in the text
    for match in re.finditer(r'\b{}\b'.format(re.escape(word_lower)), text_lower):
        span = match.span()
        info = word_info.get(word_lower, "No info available")
        words_with_info.append([
            word,
            info,
            span[0],
            span[1]
        ])

# Step 2: Count occurrences of each word in the original data
word_count = Counter(item[0] for item in words_with_info)

# Step 3: Keep only up to three entries per word
limited_entries = []
word_occurrences = Counter()

for item in words_with_info:
    word = item[0]
    if word_occurrences[word] < 3:
        limited_entries.append(item)
        word_occurrences[word] += 1

# Step 4: Add 'likely' or 'unlikely' based on the count in the original data
data_with_likelihood = [
    [item[0], item[1], item[2], item[3], 'likely' if word_count[item[0]] > 1 else 'unlikely']
    for item in limited_entries
]

# Step 5: Sort the list based on the 'end' value
sorted_data = sorted(data_with_likelihood, key=lambda x: x[3])

# Step 6: Remove duplicates from sorted_data
seen = set()
unique_sorted_data = []
for item in sorted_data:
    entry_tuple = tuple(item[:4])  # Exclude likelihood from tuple for uniqueness check
    if entry_tuple not in seen:
        seen.add(entry_tuple)
        unique_sorted_data.append(item)

# Convert unique sorted data to list of dictionaries
final_output = [
    {
        'name': item[0],
        'category': item[1],
        'start': item[2],
        'end': item[3]
    }
    for item in unique_sorted_data
]

# Print the final output
print("Unique sorted data with likelihood (in dictionary format):")
for entry in final_output:
    print(entry)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated Summary: Exemple text to summarize.
Unique sorted data with likelihood (in dictionary format):
